# Mask R-CNN - Statistics Extractor

## Initialisation

### Detecting Environment

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
print("Executing in Google Colab" if IN_COLAB else "Executing locally")

### Getting all the needed files

In [ ]:
if IN_COLAB:
    import shutil
    shutil.rmtree('sample_data/', ignore_errors=True)
    import os
    GITHUB_REPO = "https://raw.githubusercontent.com/AdrienJaugey/Custom-Mask-R-CNN-for-kidney-s-cell-recognition/master/"
    files = ['datasetTools/datasetDivider.py', 'datasetTools/AnnotationAdapter.py',
             'datasetTools/ASAPAdapter.py', 'datasetTools/LabelMeAdapter.py']
    for fileToDownload in files:
        url = GITHUB_REPO + fileToDownload
        !wget -qN $url
        if '/' in fileToDownload:
            destDir = fileToDownload.split('/')[0]
            fileName = fileToDownload.split('/')[1].replace('/', '')
            os.makedirs(destDir, exist_ok=True)
            !mv $fileName $fileToDownload

### Importing annotations file(s) to Google Colab

In [ ]:
import json
import os
import cv2
import numpy as np
from datasetTools import AnnotationAdapter as adapt
from datasetTools.datasetDivider import getBWCount

In [ ]:
if IN_COLAB:
    howToGetAnnotationFiles = "Upload" #@param ["Upload", "From Google Drive"]
    os.makedirs('annotations', exist_ok=True)

In [ ]:
if IN_COLAB and howToGetAnnotationFiles == "Upload":
    print("Please upload the annotation file(s) you want to get the stats of.")
    from google.colab import files
    import shutil
    uploaded = files.upload()
    for fileName in uploaded:
        if fileName.split('.')[-1] in adapt.ANNOTATION_FORMAT:
            shutil.move(fileName, "annotations/{}".format(fileName))
        else:
            os.remove(fileName)

In [ ]:
if IN_COLAB and howToGetAnnotationFiles == "From Google Drive":
    from google.colab import drive
    drive.mount('/content/drive')
    pathToDrive = "'/content/drive/My Drive/"
    customPathInDrive = "" #@param {type:"string"}
    annotationFilePath = "" #@param{type:"string"}
    if annotationFilePath != "":
        pathToAnnotation = pathToDrive + customPathInDrive + annotationFilePath + "'"
        tempPath = "annotations/" + annotationFilePath
        print("Copying {} to {}".format(pathToAnnotation, tempPath))
        !cp -u $pathToAnnotation $tempPath
    else:
        pathToAnnotationFolder = "/content/drive/My Drive/" + customPathInDrive
        fileList = os.listdir(pathToAnnotationFolder)
        for dataFile in fileList:
            if dataFile.split('.')[-1] in adapt.ANNOTATION_FORMAT:
                pathToFile = pathToDrive + customPathInDrive + dataFile + "'"
                tempPath = 'annotations/' + dataFile
                print("Copying {} to {}".format(pathToFile, tempPath))
                !cp -u $pathToFile $tempPath

### Declaration of functions

In [ ]:
def getMaskArea(ptsMask):
    # Formating coordinates to be usable
    ptsMask = np.double(ptsMask)
    ptsMask = np.matrix.round(ptsMask)
    ptsMask = np.int32(ptsMask)

    xs = ptsMask[:, 0]
    ys = ptsMask[:, 1]

    xMin = np.amin(xs)
    xMax = np.amax(xs)
    yMin = np.amin(ys)
    yMax = np.amax(ys)

    ptsMask[:, 0] -= np.ones((ptsMask.shape[0]), dtype=np.uint8) * xMin
    ptsMask[:, 1] -= np.ones((ptsMask.shape[0]), dtype=np.uint8) * yMin

    mask = np.uint8(np.zeros((yMax - yMin, xMax - xMin)))
    cv2.fillPoly(mask, [ptsMask], 255)

    return getBWCount(mask)[1]

In [ ]:
def getStatsFromAnnotation(annotationPath):
    adapter = None
    adapters = adapt.ANNOTATION_ADAPTERS
    adapterPriority = -1

    for a in adapters:
        if a.canRead(annotationPath):
            if a.getPriorityLevel() > adapterPriority:
                adapterPriority = a.getPriorityLevel()
                adapter = a

    masks = adapter.readFile(annotationPath)

    stats = {}

    for datasetClass, ptMask in masks:
        if type(datasetClass) is int:
            if datasetClass < len(classesInfo) and classesInfo[datasetClass]["id"] == datasetClass:
                maskClass = classesInfo[datasetClass]["name"]
            else:
                for classInfo in classesInfo:
                    if classInfo["id"] == datasetClass:
                        maskClass = classInfo["name"]
                        break
        else:
            maskClass = datasetClass

        if maskClass not in stats:
            stats[maskClass] = {"count": 0, "area": 0}

        stats[maskClass]["count"] += 1
        stats[maskClass]["area"] += getMaskArea(ptMask)
    return stats

### Link between classes names and IDs

In [ ]:
classesInfo = [
    {"id": 0, "name": "cortex"},
    {"id": 1, "name": "medullaire"},
    {"id": 2, "name": "capsule"},
    {"id": 3, "name": "tubule_sain"},
    {"id": 4, "name": "tubule_atrophique"},
    {"id": 5, "name": "nsg_complet"},
    {"id": 6, "name": "nsg_partiel"},
    {"id": 7, "name": "pac"},
    {"id": 8, "name": "vaisseau"},
    {"id": 9, "name": "artefact"},
    {"id": 10, "name": "veine"},
    {"id": 11, "name": "nsg"},
    {"id": 12, "name": "intima"},
    {"id": 13, "name": "media"}
]

## Extraction of stats

In [ ]:
saveStats = True #@param {type:"boolean"}
if saveStats:
    os.makedirs('statistics', exist_ok=True)
for annotationFile in os.listdir('annotations/'):
    if os.path.isfile(os.path.join('annotations', annotationFile)):
        imageName = annotationFile.split('.')[0]
        stats = getStatsFromAnnotation(os.path.join('annotations', annotationFile))
        print("{} statistics :".format(imageName))
        for eltClass in stats:
            print(" - {} : count = {}, area = {}".format(eltClass, stats[eltClass]["count"], stats[eltClass]["area"]))
        print()
        if saveStats:
            with open(os.path.join('statistics', '{}_stats.json'.format(imageName)), 'w') as statFile:
                json.dump(stats, statFile, indent="\t")

### Exporting results

In [ ]:
if IN_COLAB and saveStats:
        !zip -qr statistics.zip statistics/
        print("Results can be downloaded on the Files tab on the left")
        print("Zip file name is : statistics.zip")
        from google.colab import files
        files.download('statistics.zip')